#  Detecção de Fraudes no IEEE-CIS Fraud Detection com LSTM no PyTorch

Giovanna Furlan Torres

Turma 4 - Sistemas de Informação

## Problemática

Treinar e analisar uma rede neural LSTM, no Keras, utilizando o dataset IEEE-CIS Fraud Detection.

# Setup

A configuração de setup é o processo de preparar e organizar o ambiente para uso.

## Bibliotecas

Essa etapa envolve a instalação de bibliotecas e configuração de outros ajustes necessários.

In [1]:
import numpy as np
import pandas as pd
import gdown
import plotly.express as px
import plotly.graph_objects as go
from scipy.stats import zscore
from sklearn.preprocessing import MinMaxScaler
import plotly.express as px
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, Input
import numpy as np
import plotly.graph_objects as go
from sklearn.metrics import classification_report
import plotly.figure_factory as ff
from sklearn.metrics import confusion_matrix

## Importação dos dados

Para garantir o acesso aos dados por todos, sem a necessidade de permissões, foi utilizado o gdown para consumir esse dataset.

In [2]:
arquivo_IEE = "dataset1.csv"

doc_id = "1TiKpaiXm0GXuIivevga05Ad4hoBgRixH"

URL = f"https://drive.google.com/uc?id={doc_id}"

gdown.download(URL, arquivo_IEE, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1TiKpaiXm0GXuIivevga05Ad4hoBgRixH
From (redirected): https://drive.google.com/uc?id=1TiKpaiXm0GXuIivevga05Ad4hoBgRixH&confirm=t&uuid=9fe0bc53-9640-42f9-823b-a889008dc524
To: /content/dataset1.csv
100%|██████████| 683M/683M [00:08<00:00, 77.6MB/s]


'dataset1.csv'

In [3]:
arquivo_Fraudes = "dataset2.csv"

doc_id = "1u_OWAPkIdgJw1ah5xP_dGBFMSANxjxEl"

URL = f"https://drive.google.com/uc?id={doc_id}"

gdown.download(URL, arquivo_Fraudes, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1u_OWAPkIdgJw1ah5xP_dGBFMSANxjxEl
From (redirected): https://drive.google.com/uc?id=1u_OWAPkIdgJw1ah5xP_dGBFMSANxjxEl&confirm=t&uuid=ac17b8c4-7a7e-4c04-b92e-f9d8a58d3ef0
To: /content/dataset2.csv
100%|██████████| 151M/151M [00:03<00:00, 44.5MB/s]


'dataset2.csv'

In [4]:
dados_IEEE = pd.read_csv(arquivo_IEE)
dados_IEEE.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
dados_fraude = pd.read_csv(arquivo_Fraudes)
dados_fraude.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


## Observação (EXTRA):

O arquivo de dados do IEEE contém colunas relacionadas a variáveis de fraude em cartões de crédito, que atualmente apresentam valores nulos. No entanto, ao analisar o problema no Kaggle, percebi que essas colunas correspondem à base de fraudes usada na última análise ponderada. Portanto, será realizada uma concatenação entre as bases para substituir os valores `NaN` das colunas "Vxxx" pelos valores corretos.




In [6]:
# Seleciona colunas que começam com "V" seguida de números no dataset dados_fraude
colunas_fraude = [col for col in dados_fraude.columns if col.startswith('V') and col[1:].isdigit()]

# Adiciona essas colunas ao dataset dados_IEEE
dados_IEEE = pd.concat([dados_IEEE, dados_fraude[colunas_fraude]], axis=1)
dados_IEEE.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153



Além disso, como a análise estatística da base de dados de fraudes já foi realizada e está disponível no link [Fraude_Cartão_Crédito](https://github.com/furlan2803/T4M11SI/blob/main/Otimizacao%20de%20Modelos/Fraude_Cartao_Credito_Giovanna_Furlan.ipynb), este caderno apresentará apenas a análise estatística dos dados do IEEE.

## Informações Gerais

In [7]:
dados_IEEE.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 590540 entries, 0 to 590539
Columns: 422 entries, TransactionID to V28
dtypes: float64(404), int64(4), object(14)
memory usage: 1.9+ GB


### **Sumário Estatístico**

Abaixo, apresenta-se uma visão geral dos dados olímpicos:

In [8]:
num_amostras, num_features = dados_IEEE.shape

- **Número de amostras:** Indica a quantidade total de registros no conjunto de dados.

In [9]:
print(f"Número de amostras: {num_amostras}")

Número de amostras: 590540


- **Número de features:** Mostra o total de variáveis presentes em cada amostra.


In [10]:
print(f"Número de features: {num_features}")

Número de features: 422


- **Tipos de dados das features:** Exibe os tipos de dados associados a cada coluna do DataFrame.

In [11]:
print("\nTipos de dados das features:")
print(dados_IEEE.dtypes)


Tipos de dados das features:
TransactionID       int64
isFraud             int64
TransactionDT       int64
TransactionAmt    float64
ProductCD          object
                   ...   
V24               float64
V25               float64
V26               float64
V27               float64
V28               float64
Length: 422, dtype: object


- **Descrição estatística das features numéricas:** Fornece estatísticas descritivas como média, desvio padrão, valores mínimo e máximo, entre outros, para as variáveis numéricas.

In [12]:
print("\nDescrição estatística das features numéricas:")
print(dados_IEEE.describe())


Descrição estatística das features numéricas:
       TransactionID        isFraud  TransactionDT  TransactionAmt  \
count   5.905400e+05  590540.000000   5.905400e+05   590540.000000   
mean    3.282270e+06       0.034990   7.372311e+06      135.027176   
std     1.704744e+05       0.183755   4.617224e+06      239.162522   
min     2.987000e+06       0.000000   8.640000e+04        0.251000   
25%     3.134635e+06       0.000000   3.027058e+06       43.321000   
50%     3.282270e+06       0.000000   7.306528e+06       68.769000   
75%     3.429904e+06       0.000000   1.124662e+07      125.000000   
max     3.577539e+06       1.000000   1.581113e+07    31937.391000   

               card1          card2          card3          card5  \
count  590540.000000  581607.000000  588975.000000  586281.000000   
mean     9898.734658     362.555488     153.194925     199.278897   
std      4901.170153     157.793246      11.336444      41.244453   
min      1000.000000     100.000000     100.00

- **Valores ausentes por coluna:** Lista a quantidade de valores ausentes em cada coluna do DataFrame.

In [14]:
print("\nValores ausentes por coluna:")
print(dados_IEEE.isnull().sum())


Valores ausentes por coluna:
TransactionID          0
isFraud                0
TransactionDT          0
TransactionAmt         0
ProductCD              0
                   ...  
V24               305733
V25               305733
V26               305733
V27               305733
V28               305733
Length: 422, dtype: int64


- **Valores únicos em cada coluna:** Apresenta o número de valores distintos encontrados em cada coluna.

In [15]:
print("\nValores únicos em cada coluna:")
print(dados_IEEE.nunique())


Valores únicos em cada coluna:
TransactionID     590540
isFraud                2
TransactionDT     573349
TransactionAmt     20902
ProductCD              5
                   ...  
V24               275663
V25               275663
V26               275663
V27               275663
V28               275663
Length: 422, dtype: int64


## Correlação de Variáveis

A matriz de correlação mostra como duas ou mais variáveis se relacionam, com valores entre -1 (correlação negativa), 0 (sem correlação) e 1 (correlação positiva).


*   Vermelho indica uma correlação positiva.
*   Azul indica uma correlação negativa.
*   Branco indica nenhuma correlação.

A diagonal principal da matriz, que é vermelha, mostra que cada variável está perfeitamente correlacionada consigo mesma, o que é esperado.

In [16]:
# Selecionar apenas as colunas numéricas (Amostra de 10%)
df_numerico = dados_IEEE.select_dtypes(include=['float64', 'int64']).sample(frac=0.1, random_state=42)

# Calcular a matriz de correlação
correlacao = df_numerico.corr()

# Criar o heatmap
fig = px.imshow(correlacao,
                text_auto=True,
                color_continuous_scale='RdBu_r',
                title="Matriz de Correlação Simplificada")

fig.show()

# Pré-processamento

O pré-processamento de dados é uma etapa que envolve a preparação e limpeza dos dados brutos para torná-los prontos para análise.


## Tratamento de dados nulos

Calcula a mediana de cada coluna e preenche os valores nulos com essas medianas, para colunas numéricas.

Primeiro, foi calculada a porcentagem de valores ausentes (NaN) em cada coluna do conjunto de dados `dados_IEEE`. Em seguida, foram removidas as colunas que tinham mais de 50% de valores ausentes, mantendo apenas as colunas com dados mais completos. Depois, foram selecionadas apenas as colunas que contêm dados numéricos (tipos `float` e `int`). Para cada coluna numérica, foi calculada a média dos valores. Os valores zero (`0.0`) foram substituídos pela média da coluna correspondente, e os valores ausentes (NaN) também foram preenchidos com a média da coluna correspondente.

In [13]:
# Verifique a porcentagem de NaN em cada coluna
percent_nan = dados_IEEE.isna().mean() * 100

# Remova colunas que têm mais de 50% de valores NaN
threshold = 0.5
dados_IEEE.dropna(axis=1, thresh=int(threshold * len(dados_IEEE)), inplace=True)

# Selecione apenas as colunas numéricas (float e int)
numerical_columns = dados_IEEE.select_dtypes(include=['float', 'int']).columns

# Preencha os NaN e os 0.0 nas colunas numéricas com a média
for column in numerical_columns:
    mean_value = dados_IEEE[column].mean()

    # Substitui 0.0 pela média
    dados_IEEE[column].replace(0.0, mean_value, inplace=True)

    # Preencha NaN com a média
    dados_IEEE[column].fillna(mean_value, inplace=True)

dados_IEEE.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321
0,2987000,0.03499,86400,68.5,W,13926,362.555488,150.0,discover,142.0,...,39.17391,21.351473,43.319174,26.806977,109.818544,117.000000,162.153398,18.372476,42.073133,28.326584
1,2987001,0.03499,86401,29.0,W,2755,404.000000,150.0,mastercard,102.0,...,39.17391,21.351473,43.319174,26.806977,109.818544,247.606741,162.153398,18.372476,42.073133,28.326584
2,2987002,0.03499,86469,59.0,W,4663,490.000000,150.0,visa,166.0,...,39.17391,21.351473,43.319174,26.806977,109.818544,247.606741,162.153398,18.372476,42.073133,28.326584
3,2987003,0.03499,86499,50.0,W,18132,567.000000,150.0,mastercard,117.0,...,135.00000,21.351473,43.319174,26.806977,50.000000,1404.000000,790.000000,18.372476,42.073133,28.326584
4,2987004,0.03499,86506,50.0,H,4497,514.000000,150.0,mastercard,102.0,...,39.17391,21.351473,43.319174,26.806977,109.818544,247.606741,162.153398,18.372476,42.073133,28.326584


## Normalização

Primeiro, foram separadas as colunas numéricas do conjunto de dados `dados_IEEE`, selecionando aquelas com tipos `float64` e `int64`. Em seguida, foi criado um escalador (`scaler`) utilizando o `MinMaxScaler`.

Os dados numéricos foram então normalizados, aplicando o escalador para transformar os valores das colunas numéricas para uma escala entre 0 e 1. Esses passos ajudam a garantir que os dados estejam em uma escala uniforme, o que é importante para muitas técnicas de análise e modelagem.

In [14]:
# Separar colunas numéricas
colunas_numericas = dados_IEEE.select_dtypes(include=['float64', 'int64']).columns

# Criar o scaler
scaler = MinMaxScaler()

# Normalizar os dados numéricos
dados_IEEE[colunas_numericas] = scaler.fit_transform(dados_IEEE[colunas_numericas])

# Exibir o DataFrame resultante
dados_IEEE.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321
0,0.000000,0.0,0.000000e+00,0.002137,W,0.743044,0.525111,0.381679,discover,0.306569,...,0.000705,0.004372,0.005722,0.005504,0.001168,0.000871,0.001644,0.000157,0.000392,0.00026
1,0.000002,0.0,6.359409e-08,0.000900,W,0.100885,0.608000,0.381679,mastercard,0.014599,...,0.000705,0.004372,0.005722,0.005504,0.001168,0.001845,0.001644,0.000157,0.000392,0.00026
2,0.000003,0.0,4.387992e-06,0.001840,W,0.210566,0.780000,0.381679,visa,0.481752,...,0.000705,0.004372,0.005722,0.005504,0.001168,0.001845,0.001644,0.000157,0.000392,0.00026
3,0.000005,0.0,6.295815e-06,0.001558,W,0.984824,0.934000,0.381679,mastercard,0.124088,...,0.002444,0.004372,0.005722,0.005504,0.000530,0.010474,0.008019,0.000157,0.000392,0.00026
4,0.000007,0.0,6.740974e-06,0.001558,H,0.201023,0.828000,0.381679,mastercard,0.014599,...,0.000705,0.004372,0.005722,0.005504,0.001168,0.001845,0.001644,0.000157,0.000392,0.00026


## One Hot encoding

Primeiro, foram identificadas as colunas não numéricas (do tipo 'object') no conjunto de dados `dados_IEEE`. Em seguida, foi aplicado o One Hot Encoding, que transforma essas colunas categóricas em colunas binárias, criando uma nova coluna para cada categoria e atribuindo valores 0 ou 1. Após isso, o DataFrame resultante foi armazenado na variável `df_final`. Por fim, os valores booleanos (True/False) foram transformados em valores inteiros (1/0). Esses passos garantem que os dados categóricos estejam em um formato adequado para análise e modelagem, facilitando o uso no algoritmo.

In [15]:
# Identificar colunas não numéricas (do tipo 'object')
colunas_nao_numericas = dados_IEEE.select_dtypes(include=['object']).columns

# Aplicar One Hot Encoding
dados_encoded = pd.get_dummies(dados_IEEE, columns=colunas_nao_numericas, drop_first=True)
df_final = dados_encoded

# Transformar True/False em 1/0
df_final = df_final.apply(lambda x: x.astype(int) if x.dtype == 'bool' else x)
df_final.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,card1,card2,card3,card5,addr1,addr2,...,P_emaildomain_yahoo.de,P_emaildomain_yahoo.es,P_emaildomain_yahoo.fr,P_emaildomain_ymail.com,M1_T,M2_T,M3_T,M4_M1,M4_M2,M6_T
0,0.000000,0.0,0.000000e+00,0.002137,0.743044,0.525111,0.381679,0.306569,0.488636,0.836957,...,0,0,0,0,1,1,1,0,1,1
1,0.000002,0.0,6.359409e-08,0.000900,0.100885,0.608000,0.381679,0.014599,0.511364,0.836957,...,0,0,0,0,0,0,0,0,0,1
2,0.000003,0.0,4.387992e-06,0.001840,0.210566,0.780000,0.381679,0.481752,0.522727,0.836957,...,0,0,0,0,1,1,1,0,0,0
3,0.000005,0.0,6.295815e-06,0.001558,0.984824,0.934000,0.381679,0.124088,0.854545,0.836957,...,0,0,0,0,0,0,0,0,0,0
4,0.000007,0.0,6.740974e-06,0.001558,0.201023,0.828000,0.381679,0.014599,0.727273,0.836957,...,0,0,0,0,0,0,0,0,0,0


# Modelo

Criou-se um modelo utilizando a arquitetura LSTM (Long Short-Term Memory). Esse tipo de rede neural é especialmente utilizada para trabalhar com dados sequenciais e séries temporais, pois consegue capturar dependências de longo prazo. O modelo foi treinado com os dados pré-processados.

In [16]:
# Selecionando 10% do dataset
df_half = df_final.sample(frac=0.1, random_state=42)

# Definindo o número de timesteps e features
timesteps = 1
num_features = df_half.shape[1] - 1

# Preparar os dados
X = df_half.drop(columns=['isFraud']).values
y = df_half['isFraud'].values

# Redimensionar para o formato
X = X.reshape((X.shape[0], timesteps, num_features))

# Balanceamento com SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X.reshape(X.shape[0], -1), y)  # reshape para aplicar SMOTE
X_resampled = X_resampled.reshape((X_resampled.shape[0], timesteps, num_features))

# Divisão entre treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42)


In [17]:
# Definindo o modelo
model = Sequential()

# Camada de entrada
model.add(Input(shape=(timesteps, num_features)))

# Primeira camada LSTM
model.add(LSTM(128, activation='tanh', return_sequences=True))
model.add(Dropout(0.2))

# Segunda camada LSTM
model.add(LSTM(64, activation='tanh', return_sequences=False))
model.add(Dropout(0.2))

# Camada totalmente conectada
model.add(Dense(32, activation='relu'))

# Camada de saída
model.add(Dense(1, activation='sigmoid'))

# Compilando o modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [18]:
# Treinamento
history = model.fit(X_train, y_train, epochs=30, batch_size=64, validation_data=(X_test, y_test))

# Ajustando o threshold
y_pred_prob = model.predict(X_test)
threshold = 0.3  # Reduzindo o threshold para aumentar o recall
y_pred = (y_pred_prob >= threshold).astype(int)

Epoch 1/30
1246/1246 ━━━━━━━━━━━━━━━━━━━━ 22s 12ms/step - accuracy: 0.7300 - loss: 0.5319 - val_accuracy: 0.7829 - val_loss: 0.4529
Epoch 2/30
1246/1246 ━━━━━━━━━━━━━━━━━━━━ 26s 17ms/step - accuracy: 0.7918 - loss: 0.4402 - val_accuracy: 0.8127 - val_loss: 0.4087
Epoch 3/30
1246/1246 ━━━━━━━━━━━━━━━━━━━━ 17s 14ms/step - accuracy: 0.8165 - loss: 0.4017 - val_accuracy: 0.8291 - val_loss: 0.3757
Epoch 4/30
1246/1246 ━━━━━━━━━━━━━━━━━━━━ 23s 16ms/step - accuracy: 0.8338 - loss: 0.3660 - val_accuracy: 0.8552 - val_loss: 0.3323
Epoch 5/30
1246/1246 ━━━━━━━━━━━━━━━━━━━━ 18s 14ms/step - accuracy: 0.8536 - loss: 0.3323 - val_accuracy: 0.8656 - val_loss: 0.3092
Epoch 6/30
1246/1246 ━━━━━━━━━━━━━━━━━━━━ 17s 13ms/step - accuracy: 0.8683 - loss: 0.3036 - val_accuracy: 0.8813 - val_loss: 0.2790
Epoch 7/30
1246/1246 ━━━━━━━━━━━━━━━━━━━━ 23s 16ms/step - accuracy: 0.8838 - loss: 0.2735 - val_accuracy: 0.8964 - val_loss: 0.2526
Epoch 8/30
1246/1246 ━━━━━━━━━━━━━━━━━━━━ 24s 19ms/step - accuracy: 0.8933 -

## Análise do Modelo

### Gráfico de Perda (Loss)

Este gráfico mostra a perda do modelo ao longo das épocas de treinamento e validação. Se o gráfico indica que a perda está diminuindo gradualmente até estabilizar, é um bom sinal de que o modelo está aprendendo. Contudo, se a curva de validação começar a subir enquanto a de treinamento continua caindo, isso pode indicar overfitting, o que significa que o modelo está se ajustando demais aos dados de treino e não generaliza bem para novos dados.

In [19]:
# Criando gráficos de perda e precisão
fig = go.Figure()

# Curva de perda
fig.add_trace(go.Scatter(
    x=list(range(1, len(history.history['loss']) + 1)),
    y=history.history['loss'],
    mode='lines+markers',
    name='Perda de Treinamento',
    line=dict(color='blue')
))

fig.add_trace(go.Scatter(
    x=list(range(1, len(history.history['val_loss']) + 1)),
    y=history.history['val_loss'],
    mode='lines+markers',
    name='Perda de Validação',
    line=dict(color='red')
))

fig.update_layout(
    title='Curvas de Perda',
    xaxis_title='Épocas',
    yaxis_title='Perda',
    legend_title='Tipo'
)

fig.show()

### Gráfico de Precisão (Accuracy)

O gráfico de precisão mostra a evolução da acurácia ao longo das épocas. Uma acurácia crescente e estável sugere que o modelo está aprendendo corretamente. No entanto, uma acurácia muito alta desde o início pode indicar overfitting, especialmente se houver um grande desbalanceamento entre as classes, como é o caso com fraudes.

In [20]:
# Criando gráfico de precisão
fig_accuracy = go.Figure()

fig_accuracy.add_trace(go.Scatter(
    x=list(range(1, len(history.history['accuracy']) + 1)),
    y=history.history['accuracy'],
    mode='lines+markers',
    name='Precisão de Treinamento',
    line=dict(color='green')
))

fig_accuracy.add_trace(go.Scatter(
    x=list(range(1, len(history.history['val_accuracy']) + 1)),
    y=history.history['val_accuracy'],
    mode='lines+markers',
    name='Precisão de Validação',
    line=dict(color='orange')
))

fig_accuracy.update_layout(
    title='Curvas de Precisão',
    xaxis_title='Épocas',
    yaxis_title='Precisão',
    legend_title='Tipo'
)

fig_accuracy.show()


### Matriz de Confusão

A matriz de confusão avalia a performance do modelo em termos de classificações corretas e incorretas. Ela mostra os verdadeiros positivos (fraudes detectadas corretamente), verdadeiros negativos (transações legítimas classificadas corretamente), falsos positivos (transações legítimas classificadas como fraude) e falsos negativos (fraudes não detectadas). A análise desses números pode revelar como o modelo está performando em cada classe, o que é importante em cenários de detecção de fraudes, onde erros podem ter grandes implicações.

In [22]:
# Gerando a matriz de confusão
cm = confusion_matrix(y_test, y_pred)

# Criando a figura da matriz de confusão com Plotly
fig = ff.create_annotated_heatmap(z=cm,
                                  x=['Predito: Não Fraude', 'Predito: Fraude'],
                                  y=['Verdadeiro: Não Fraude', 'Verdadeiro: Fraude'],
                                  colorscale='Blues')

# Atualizando o layout
fig.update_layout(title='Matriz de Confusão', xaxis_title='Classe Predita', yaxis_title='Classe Verdadeira')

# Exibindo o gráfico
fig.show()

### Classification Report

O relatório de classificação exibe as principais métricas de avaliação: precisão, recall e F1-score, além da acurácia geral.

**Precisão (Precision):** Indica a porcentagem de transações classificadas como fraude que realmente são fraudes. Uma precisão alta é importante para evitar falsos positivos (transações legítimas classificadas como fraude).

**Recall:** Mede a capacidade do modelo de identificar todas as fraudes (verdadeiros positivos). Um recall alto é importante para reduzir falsos negativos (fraudes não detectadas).

**F1-Score:** Uma métrica combinada de precisão e recall, útil especialmente quando há um desbalanceamento nas classes.

**Acurácia:** No caso de detecção de fraudes, o foco deve estar mais na precisão e no recall da classe minoritária (fraude), devido ao desbalanceamento dos dados.

In [24]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.99      0.89      0.94     17234
         1.0       0.90      0.99      0.95     16933

    accuracy                           0.94     34167
   macro avg       0.95      0.94      0.94     34167
weighted avg       0.95      0.94      0.94     34167



# Conclusão : Resultado do Modelo

Os gráficos de perda e precisão indicam que o modelo está aprendendo bem ao longo das épocas. A curva de perda de treinamento mostra uma redução contínua, sugerindo que o modelo está se ajustando aos dados de treinamento. Já a perda de validação, apesar de também diminuir, tem uma convergência mais suave, sugerindo uma generalização adequada, mas que ainda pode ser otimizada com técnicas como regularização ou aumento de dados.


As métricas apresentadas indicam que o modelo possui um desempenho bom, especialmente em termos de precisão e recall, com uma acurácia geral de 94%. No geral, o modelo se sai bem tanto ao identificar transações normais quanto fraudes, mas há alguns pontos que merecem destaque:

1. **Precisão**: O modelo tem uma alta precisão para a classe 0 (99%), o que significa que ele raramente classifica incorretamente transações normais como fraudes. Já para a classe 1, a precisão é de 90%, indicando que 10% das transações classificadas como fraudes, na verdade, não são. Ainda assim, esse valor é bastante aceitável para cenários de fraude, onde um equilíbrio entre precisão e recall é crucial.

2. **Recall**: O recall para a classe 0 é de 89%, o que sugere que o modelo não consegue identificar todas as transações normais, com algumas sendo incorretamente classificadas como fraudes. Entretanto, o recall de 99% para a classe 1 indica que o modelo é excelente em detectar quase todas as fraudes, o que é essencial em um cenário de detecção de fraudes.

3. **F1-Score**: Os F1-Scores para ambas as classes (0.94 para transações normais e 0.95 para fraudes) mostram que o modelo consegue equilibrar bem precisão e recall, especialmente na detecção das fraudes, o que é positivo.

Em resumo, as métricas indicam um bom desempenho do modelo, e que com base nas análises seu desempenho foi bom e sem indícios de overfiting em seu teste.

# Observações Gerais (Análise Extra)

No modelo de detecção de fraudes com LSTM, foi observado que os dados utilizados não possuem características sequenciais explícitas, como uma série temporal ou uma variável de data que pudesse servir de base para validar a aplicação de uma rede LSTM. Assim, não é possível garantir que a LSTM seja a melhor abordagem, dado que ela é ideal para modelar dados com dependência temporal, o que não é o caso aqui.

O conjunto de dados possui em média 590 mil linhas, o que apresentou um desafio de capacidade de memória (RAM). Para contornar essa limitação, foram testadas diferentes proporções do dataset para avaliar o desempenho do modelo. Inicialmente, utilizou-se 100% dos dados, o que não foi viável. Com 10% do dataset, o modelo foi treinado em cerca de 30 minutos, apresentando bons resultados. Ao usar 50% dos dados, o tempo estimado de treinamento foi de 3 horas. Dessa forma, optou-se por utilizar 10% do dataset como a fração ponderada para o treinamento final.

Além disso, foram realizados testes com o modelo sem balanceamento de classes, com 20 épocas de treinamento e apenas uma camada LSTM, o que resultou em overfitting, já que a acurácia nos testes já iniciava a primeira época com 96%, indicando que o modelo estava se ajustando demais aos dados de treino. Para mitigar isso, foram adotadas as seguintes estratégias:

- **SMOTE**: Aplicado para balancear as classes antes da divisão entre treino e teste, reduzindo o viés do modelo em relação à classe majoritária.
- **Camadas LSTM**: O número de neurônios nas camadas LSTM foi aumentado para melhorar a capacidade do modelo de capturar padrões nos dados.
- **Epochs**: O número de épocas foi ampliado para 30, permitindo ao modelo mais tempo para ajustar os pesos sem cair em overfitting tão cedo.
- **Threshold**: O limiar de decisão foi ajustado para 0.3, melhorando a detecção de fraudes ao equilibrar a precisão e o recall do modelo.

Essas alterações resultaram em um melhor modelo, capaz de lidar melhor com o desbalanceamento de classes e com um desempenho mais consistente nas previsões.